In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OneHotEncoder
import pandas as pd
import tensorflow as tf
from pathlib import Path



In [72]:
# Import our input dataset
data = Path('Resources_k/fire_size_bins.csv')
df = pd.read_csv(data)
df.head()

,Unnamed: 0,fire_id,fire_size,fire_cause,latitude,longitude,state,discovery_month,Temp_pre_30,Temp_pre_15,Temp_pre_7,Wind_pre_30,Wind_pre_15,Wind_pre_7,Hum_pre_30,Hum_pre_15,Hum_pre_7,year,putout_time,fire_size_bin
0,0,3,1.00,Debris Burning,39.641400,-119.308300,NV,Jun,16.275967,18.996181,18.142564,4.054982,3.398329,3.671282,44.778429,37.140811,35.353846,2005,0,Teacup
1,1,24,40.00,Arson,31.435181,-88.999489,MS,Apr,13.468619,15.067227,15.604790,2.038268,1.737921,1.775904,57.997207,56.747191,59.614458,1999,0,Mini
2,3,31,1.20,Debris Burning,48.833000,-99.783600,ND,Apr,-0.891635,0.372659,-4.273834,5.800667,6.012852,6.658621,77.575012,75.963981,71.173116,2007,0,Teacup
3,4,35,30.18,Debris Burning,31.259000,-84.895600,GA,Oct,20.079480,17.722714,18.188679,3.659840,3.366443,2.211429,67.551783,61.733788,60.328571,2000,0,Mini
4,5,36,1420.00,Lightning,33.241800,-104.912200,NM,Jul,31.055859,32.523438,34.893333,4.026367,3.844922,3.695833,28.783203,25.789062,18.208333,1994,1,Large


In [73]:
df.drop(columns=["Unnamed: 0", "fire_id", "fire_size", "latitude", "longitude", "state", "year"],inplace=True)

In [74]:
df

,fire_cause,discovery_month,Temp_pre_30,Temp_pre_15,Temp_pre_7,Wind_pre_30,Wind_pre_15,Wind_pre_7,Hum_pre_30,Hum_pre_15,Hum_pre_7,putout_time,fire_size_bin
0,Debris Burning,Jun,16.275967,18.996181,18.142564,4.054982,3.398329,3.671282,44.778429,37.140811,35.353846,0,Teacup
1,Arson,Apr,13.468619,15.067227,15.604790,2.038268,1.737921,1.775904,57.997207,56.747191,59.614458,0,Mini
2,Debris Burning,Apr,-0.891635,0.372659,-4.273834,5.800667,6.012852,6.658621,77.575012,75.963981,71.173116,0,Teacup
3,Debris Burning,Oct,20.079480,17.722714,18.188679,3.659840,3.366443,2.211429,67.551783,61.733788,60.328571,0,Mini
4,Lightning,Jul,31.055859,32.523438,34.893333,4.026367,3.844922,3.695833,28.783203,25.789062,18.208333,1,Large
...,...,...,...,...,...,...,...,...,...,...,...,...,...
13133,Utilities,Aug,30.731860,30.545367,28.719917,3.136761,3.238803,3.224274,50.089157,49.416505,55.426471,3,Large
13134,Utilities,Oct,24.642268,23.713390,24.221869,1.529850,1.576828,1.563817,62.848172,60.641975,63.196819,11,Large
13135,Accidental,Sep,15.546194,12.890633,10.734328,2.608150,2.486802,1.835821,55.009259,62.368700,67.266304,39,XL
13136,Debris Burning,Mar,2.275974,7.360185,7.678571,4.428757,4.197593,3.803571,51.676681,46.933399,42.638384,5,Large


In [75]:
# Generate our categorical variable list
#cat = df.dtypes[df.dtypes == "object"].index.tolist()
cat=['fire_cause', 'discovery_month']

# Check the number of unique values in each column
df[cat].nunique()



fire_cause          5
discovery_month    12
dtype: int64

In [76]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(df[cat]))

# Add the encoded variable names to the DataFrame
encode_df.columns = enc.get_feature_names(cat)
encode_df.head()

,fire_cause_Accidental,fire_cause_Arson,fire_cause_Debris Burning,fire_cause_Lightning,fire_cause_Utilities,discovery_month_Apr,discovery_month_Aug,discovery_month_Dec,discovery_month_Feb,discovery_month_Jan,discovery_month_Jul,discovery_month_Jun,discovery_month_Mar,discovery_month_May,discovery_month_Nov,discovery_month_Oct,discovery_month_Sep
0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [77]:
# Merge one-hot encoded features and drop the originals
df = df.merge(encode_df,left_index=True, right_index=True)
df = df.drop(cat,1)
df.head()

C:\Users\KATLIN\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  This is separate from the ipykernel package so we can avoid doing imports until


,Temp_pre_30,Temp_pre_15,Temp_pre_7,Wind_pre_30,Wind_pre_15,Wind_pre_7,Hum_pre_30,Hum_pre_15,Hum_pre_7,putout_time,...,discovery_month_Dec,discovery_month_Feb,discovery_month_Jan,discovery_month_Jul,discovery_month_Jun,discovery_month_Mar,discovery_month_May,discovery_month_Nov,discovery_month_Oct,discovery_month_Sep
0,16.275967,18.996181,18.142564,4.054982,3.398329,3.671282,44.778429,37.140811,35.353846,0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,13.468619,15.067227,15.604790,2.038268,1.737921,1.775904,57.997207,56.747191,59.614458,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,-0.891635,0.372659,-4.273834,5.800667,6.012852,6.658621,77.575012,75.963981,71.173116,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,20.079480,17.722714,18.188679,3.659840,3.366443,2.211429,67.551783,61.733788,60.328571,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,31.055859,32.523438,34.893333,4.026367,3.844922,3.695833,28.783203,25.789062,18.208333,1,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [78]:
df.columns

Index(['Temp_pre_30', 'Temp_pre_15', 'Temp_pre_7', 'Wind_pre_30',
       'Wind_pre_15', 'Wind_pre_7', 'Hum_pre_30', 'Hum_pre_15', 'Hum_pre_7',
       'putout_time', 'fire_size_bin', 'fire_cause_Accidental',
       'fire_cause_Arson', 'fire_cause_Debris Burning', 'fire_cause_Lightning',
       'fire_cause_Utilities', 'discovery_month_Apr', 'discovery_month_Aug',
       'discovery_month_Dec', 'discovery_month_Feb', 'discovery_month_Jan',
       'discovery_month_Jul', 'discovery_month_Jun', 'discovery_month_Mar',
       'discovery_month_May', 'discovery_month_Nov', 'discovery_month_Oct',
       'discovery_month_Sep'],
      dtype='object')

In [79]:
# # Import label encoder
# from sklearn import preprocessing
 
# # label_encoder object knows how to understand word labels.
# label_encoder = preprocessing.LabelEncoder()
 
# # Encode labels in column 'fire_size_bin.
# df['fire_size_bin']= label_encoder.fit_transform(df['fire_size_bin'])
 
# df['fire_size_bin'].unique()

In [80]:
df["size_bin_no"]=''
df

,Temp_pre_30,Temp_pre_15,Temp_pre_7,Wind_pre_30,Wind_pre_15,Wind_pre_7,Hum_pre_30,Hum_pre_15,Hum_pre_7,putout_time,...,discovery_month_Feb,discovery_month_Jan,discovery_month_Jul,discovery_month_Jun,discovery_month_Mar,discovery_month_May,discovery_month_Nov,discovery_month_Oct,discovery_month_Sep,size_bin_no
0,16.275967,18.996181,18.142564,4.054982,3.398329,3.671282,44.778429,37.140811,35.353846,0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,
1,13.468619,15.067227,15.604790,2.038268,1.737921,1.775904,57.997207,56.747191,59.614458,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,
2,-0.891635,0.372659,-4.273834,5.800667,6.012852,6.658621,77.575012,75.963981,71.173116,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,
3,20.079480,17.722714,18.188679,3.659840,3.366443,2.211429,67.551783,61.733788,60.328571,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,
4,31.055859,32.523438,34.893333,4.026367,3.844922,3.695833,28.783203,25.789062,18.208333,1,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13133,30.731860,30.545367,28.719917,3.136761,3.238803,3.224274,50.089157,49.416505,55.426471,3,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,
13134,24.642268,23.713390,24.221869,1.529850,1.576828,1.563817,62.848172,60.641975,63.196819,11,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,
13135,15.546194,12.890633,10.734328,2.608150,2.486802,1.835821,55.009259,62.368700,67.266304,39,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,
13136,2.275974,7.360185,7.678571,4.428757,4.197593,3.803571,51.676681,46.933399,42.638384,5,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,


In [81]:
df.loc[(df['fire_size_bin']=="Teacup"),"size_bin_no"] = 0
df.loc[(df['fire_size_bin']=="Toy"),"size_bin_no"] = 1
df.loc[(df['fire_size_bin']=="Mini"),"size_bin_no"] = 2
df.loc[(df['fire_size_bin']=="Medium"),"size_bin_no"] = 3
df.loc[(df['fire_size_bin']=="Large"),"size_bin_no"] = 4
df.loc[(df['fire_size_bin']=="XL"),"size_bin_no"] = 5
df


,Temp_pre_30,Temp_pre_15,Temp_pre_7,Wind_pre_30,Wind_pre_15,Wind_pre_7,Hum_pre_30,Hum_pre_15,Hum_pre_7,putout_time,...,discovery_month_Feb,discovery_month_Jan,discovery_month_Jul,discovery_month_Jun,discovery_month_Mar,discovery_month_May,discovery_month_Nov,discovery_month_Oct,discovery_month_Sep,size_bin_no
0,16.275967,18.996181,18.142564,4.054982,3.398329,3.671282,44.778429,37.140811,35.353846,0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0
1,13.468619,15.067227,15.604790,2.038268,1.737921,1.775904,57.997207,56.747191,59.614458,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2
2,-0.891635,0.372659,-4.273834,5.800667,6.012852,6.658621,77.575012,75.963981,71.173116,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,20.079480,17.722714,18.188679,3.659840,3.366443,2.211429,67.551783,61.733788,60.328571,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2
4,31.055859,32.523438,34.893333,4.026367,3.844922,3.695833,28.783203,25.789062,18.208333,1,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13133,30.731860,30.545367,28.719917,3.136761,3.238803,3.224274,50.089157,49.416505,55.426471,3,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4
13134,24.642268,23.713390,24.221869,1.529850,1.576828,1.563817,62.848172,60.641975,63.196819,11,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,4
13135,15.546194,12.890633,10.734328,2.608150,2.486802,1.835821,55.009259,62.368700,67.266304,39,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,5
13136,2.275974,7.360185,7.678571,4.428757,4.197593,3.803571,51.676681,46.933399,42.638384,5,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,4


In [82]:
df['size_bin_no']=df.size_bin_no.astype(int)

In [83]:
df.drop(['fire_size_bin'], axis=1, inplace=True)


In [84]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13138 entries, 0 to 13137
Data columns (total 28 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Temp_pre_30                13138 non-null  float64
 1   Temp_pre_15                13138 non-null  float64
 2   Temp_pre_7                 13138 non-null  float64
 3   Wind_pre_30                13138 non-null  float64
 4   Wind_pre_15                13138 non-null  float64
 5   Wind_pre_7                 13138 non-null  float64
 6   Hum_pre_30                 13138 non-null  float64
 7   Hum_pre_15                 13138 non-null  float64
 8   Hum_pre_7                  13138 non-null  float64
 9   putout_time                13138 non-null  int64  
 10  fire_cause_Accidental      13138 non-null  float64
 11  fire_cause_Arson           13138 non-null  float64
 12  fire_cause_Debris Burning  13138 non-null  float64
 13  fire_cause_Lightning       13138 non-null  flo

In [95]:
df.columns

Index(['Temp_pre_30', 'Temp_pre_15', 'Temp_pre_7', 'Wind_pre_30',
       'Wind_pre_15', 'Wind_pre_7', 'Hum_pre_30', 'Hum_pre_15', 'Hum_pre_7',
       'putout_time', 'fire_cause_Accidental', 'fire_cause_Arson',
       'fire_cause_Debris Burning', 'fire_cause_Lightning',
       'fire_cause_Utilities', 'discovery_month_Apr', 'discovery_month_Aug',
       'discovery_month_Dec', 'discovery_month_Feb', 'discovery_month_Jan',
       'discovery_month_Jul', 'discovery_month_Jun', 'discovery_month_Mar',
       'discovery_month_May', 'discovery_month_Nov', 'discovery_month_Oct',
       'discovery_month_Sep', 'size_bin_no'],
      dtype='object')

In [106]:
# Remove loan status target from features data
target=df.size_bin_no
y=target
# X = df.loc[:, df.columns !=target]
X=df.drop(columns=["size_bin_no", ]).values

# Split training/test datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [107]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 = 60


nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
#nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_27 (Dense)            (None, 60)                1680      
                                                                 
 dense_28 (Dense)            (None, 1)                 61        
                                                                 
Total params: 1,741
Trainable params: 1,741
Non-trainable params: 0
_________________________________________________________________


In [108]:
# Compile the model
nn.compile(loss='categorical_crossentropy', optimizer="adam", metrics=["accuracy"])

In [109]:
## Import checkpoint dependencies
# import os
# from tensorflow.keras.callbacks import ModelCheckpoint

# # Define the checkpoint path and filenames
# os.makedirs("checkpoints/",exist_ok=True)
# checkpoint_path = "checkpoints/weights.{epoch:02d}.hdf5"

# # Create a callback that saves the model's weights every 5 epoch
# cp_callback = ModelCheckpoint(
#     filepath=checkpoint_path,
#     verbose=5,
#     save_weights_only=True,
#     save_freq='epoch')

# Train the model
# fit_model = nn.fit(X_train_scaled,y_train,epochs=100,callbacks=[cp_callback])
fit_model = nn.fit(X_train_scaled,y_train,epochs=50)

# # Restore the model weights
# nn.load_weights("checkpoints/weights.100.hdf5")


Epoch 1/50
308/308 [==============================] - 1s 2ms/step - loss: 0.0000e+00 - accuracy: 0.2845
Epoch 2/50
308/308 [==============================] - 0s 1ms/step - loss: 0.0000e+00 - accuracy: 0.3130
Epoch 3/50
308/308 [==============================] - 0s 2ms/step - loss: 0.0000e+00 - accuracy: 0.3189
Epoch 4/50
308/308 [==============================] - 0s 1ms/step - loss: 0.0000e+00 - accuracy: 0.3200
Epoch 5/50
308/308 [==============================] - 0s 1ms/step - loss: 0.0000e+00 - accuracy: 0.3217
Epoch 6/50
308/308 [==============================] - 0s 1ms/step - loss: 0.0000e+00 - accuracy: 0.3225
Epoch 7/50
308/308 [==============================] - 0s 1ms/step - loss: 0.0000e+00 - accuracy: 0.3235
Epoch 8/50
308/308 [==============================] - 0s 1ms/step - loss: 0.0000e+00 - accuracy: 0.3241
Epoch 9/50
308/308 [==============================] - 0s 1ms/step - loss: 0.0000e+00 - accuracy: 0.3243
Epoch 10/50
308/308 [==============================] - 0s 1ms/st

In [110]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

103/103 - 0s - loss: 0.0000e+00 - accuracy: 0.3315 - 199ms/epoch - 2ms/step
Loss: 0.0, Accuracy: 0.3315068483352661


---------------------

In [50]:
# Import our input dataset
data = Path('Resources_k/fire_size_bins.csv')
df = pd.read_csv(data)
df.head()

,Unnamed: 0,fire_id,fire_size,fire_cause,latitude,longitude,state,discovery_month,Temp_pre_30,Temp_pre_15,Temp_pre_7,Wind_pre_30,Wind_pre_15,Wind_pre_7,Hum_pre_30,Hum_pre_15,Hum_pre_7,year,putout_time,fire_size_bin
0,0,3,1.00,Debris Burning,39.641400,-119.308300,NV,Jun,16.275967,18.996181,18.142564,4.054982,3.398329,3.671282,44.778429,37.140811,35.353846,2005,0,Teacup
1,1,24,40.00,Arson,31.435181,-88.999489,MS,Apr,13.468619,15.067227,15.604790,2.038268,1.737921,1.775904,57.997207,56.747191,59.614458,1999,0,Mini
2,3,31,1.20,Debris Burning,48.833000,-99.783600,ND,Apr,-0.891635,0.372659,-4.273834,5.800667,6.012852,6.658621,77.575012,75.963981,71.173116,2007,0,Teacup
3,4,35,30.18,Debris Burning,31.259000,-84.895600,GA,Oct,20.079480,17.722714,18.188679,3.659840,3.366443,2.211429,67.551783,61.733788,60.328571,2000,0,Mini
4,5,36,1420.00,Lightning,33.241800,-104.912200,NM,Jul,31.055859,32.523438,34.893333,4.026367,3.844922,3.695833,28.783203,25.789062,18.208333,1994,1,Large


In [51]:
df.drop(columns=["Unnamed: 0", "fire_id", "fire_size", "latitude", "longitude", "state", "year"],inplace=True)
df

,fire_cause,discovery_month,Temp_pre_30,Temp_pre_15,Temp_pre_7,Wind_pre_30,Wind_pre_15,Wind_pre_7,Hum_pre_30,Hum_pre_15,Hum_pre_7,putout_time,fire_size_bin
0,Debris Burning,Jun,16.275967,18.996181,18.142564,4.054982,3.398329,3.671282,44.778429,37.140811,35.353846,0,Teacup
1,Arson,Apr,13.468619,15.067227,15.604790,2.038268,1.737921,1.775904,57.997207,56.747191,59.614458,0,Mini
2,Debris Burning,Apr,-0.891635,0.372659,-4.273834,5.800667,6.012852,6.658621,77.575012,75.963981,71.173116,0,Teacup
3,Debris Burning,Oct,20.079480,17.722714,18.188679,3.659840,3.366443,2.211429,67.551783,61.733788,60.328571,0,Mini
4,Lightning,Jul,31.055859,32.523438,34.893333,4.026367,3.844922,3.695833,28.783203,25.789062,18.208333,1,Large
...,...,...,...,...,...,...,...,...,...,...,...,...,...
13133,Utilities,Aug,30.731860,30.545367,28.719917,3.136761,3.238803,3.224274,50.089157,49.416505,55.426471,3,Large
13134,Utilities,Oct,24.642268,23.713390,24.221869,1.529850,1.576828,1.563817,62.848172,60.641975,63.196819,11,Large
13135,Accidental,Sep,15.546194,12.890633,10.734328,2.608150,2.486802,1.835821,55.009259,62.368700,67.266304,39,XL
13136,Debris Burning,Mar,2.275974,7.360185,7.678571,4.428757,4.197593,3.803571,51.676681,46.933399,42.638384,5,Large


In [53]:
# Generate our categorical variable list
#cat = df.dtypes[df.dtypes == "object"].index.tolist()
cat=['fire_cause', 'discovery_month']

# Check the number of unique values in each column
df[cat].nunique()

fire_cause          5
discovery_month    12
dtype: int64

In [54]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(df[cat]))

# Add the encoded variable names to the DataFrame
encode_df.columns = enc.get_feature_names(cat)
encode_df.head()

,fire_cause_Accidental,fire_cause_Arson,fire_cause_Debris Burning,fire_cause_Lightning,fire_cause_Utilities,discovery_month_Apr,discovery_month_Aug,discovery_month_Dec,discovery_month_Feb,discovery_month_Jan,discovery_month_Jul,discovery_month_Jun,discovery_month_Mar,discovery_month_May,discovery_month_Nov,discovery_month_Oct,discovery_month_Sep
0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [55]:
# Merge one-hot encoded features and drop the originals
df = df.merge(encode_df,left_index=True, right_index=True)
df = df.drop(cat,1)
df.head()

C:\Users\KATLIN\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  This is separate from the ipykernel package so we can avoid doing imports until


,Temp_pre_30,Temp_pre_15,Temp_pre_7,Wind_pre_30,Wind_pre_15,Wind_pre_7,Hum_pre_30,Hum_pre_15,Hum_pre_7,putout_time,...,discovery_month_Dec,discovery_month_Feb,discovery_month_Jan,discovery_month_Jul,discovery_month_Jun,discovery_month_Mar,discovery_month_May,discovery_month_Nov,discovery_month_Oct,discovery_month_Sep
0,16.275967,18.996181,18.142564,4.054982,3.398329,3.671282,44.778429,37.140811,35.353846,0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,13.468619,15.067227,15.604790,2.038268,1.737921,1.775904,57.997207,56.747191,59.614458,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,-0.891635,0.372659,-4.273834,5.800667,6.012852,6.658621,77.575012,75.963981,71.173116,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,20.079480,17.722714,18.188679,3.659840,3.366443,2.211429,67.551783,61.733788,60.328571,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,31.055859,32.523438,34.893333,4.026367,3.844922,3.695833,28.783203,25.789062,18.208333,1,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [56]:
# Remove loan status target from features data
target=df.fire_size_bin
y=target
# X = df.loc[:, df.columns !=target]
X=df.drop(columns=['fire_size_bin']).values

# Split training/test datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [57]:
from imblearn.ensemble import BalancedRandomForestClassifier
model = BalancedRandomForestClassifier(n_estimators=128, random_state=78) 

#Fitting the model
model = model.fit(X_train, y_train)

# Making predictions using the testing data.
y_pred = model.predict(X_test)

In [111]:
# Calculating the accuracy score.
from sklearn.metrics import balanced_accuracy_score
acc_score = balanced_accuracy_score(y_test, y_pred)
acc_score

ValueError: Mix of label input types (string and number)

In [59]:
from sklearn.metrics import confusion_matrix
# Calculating the confusion matrix
cm = confusion_matrix(y_test, y_pred)
cm_df = pd.DataFrame(
    cm, index=["Actual Teacup", "Actual Toy", "Actual Mini", "Actual Medium", "Actual Large", "Actual XL"], columns=["Predicted Teacup", "Predicted Toy", "Predicted Mini", "Predicted Medium", "Predicted Large", "Predicted XL"]
)
cm_df

,Predicted Teacup,Predicted Toy,Predicted Mini,Predicted Medium,Predicted Large,Predicted XL
Actual Teacup,204,30,3,13,4,148
Actual Toy,49,76,50,34,49,25
Actual Mini,48,66,122,119,111,11
Actual Medium,70,202,308,546,346,20
Actual Large,19,47,72,90,83,5
Actual XL,72,10,1,6,4,222


In [60]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))


                   pre       rec       spe        f1       geo       iba       sup

      Large       0.44      0.51      0.91      0.47      0.68      0.44       402
     Medium       0.18      0.27      0.88      0.21      0.49      0.22       283
       Mini       0.22      0.26      0.85      0.24      0.47      0.20       477
     Teacup       0.68      0.37      0.85      0.47      0.56      0.30      1492
        Toy       0.14      0.26      0.83      0.18      0.47      0.20       316
         XL       0.52      0.70      0.93      0.60      0.81      0.64       315

avg / total       0.47      0.38      0.87      0.40      0.57      0.32      3285



In [61]:
# Displaying Easy Ensemble Classifier
print("Balanced Random Forest Clasifier")
print("Confusion Matrix")
display(cm_df)
print(f"Accuracy Score : {acc_score}")
print("Classification Report")
print(classification_report_imbalanced(y_test, y_pred))

Balanced Random Forest Clasifier
Confusion Matrix


,Predicted Teacup,Predicted Toy,Predicted Mini,Predicted Medium,Predicted Large,Predicted XL
Actual Teacup,204,30,3,13,4,148
Actual Toy,49,76,50,34,49,25
Actual Mini,48,66,122,119,111,11
Actual Medium,70,202,308,546,346,20
Actual Large,19,47,72,90,83,5
Actual XL,72,10,1,6,4,222


Accuracy Score : 0.39419183295250976
Classification Report
                   pre       rec       spe        f1       geo       iba       sup

      Large       0.44      0.51      0.91      0.47      0.68      0.44       402
     Medium       0.18      0.27      0.88      0.21      0.49      0.22       283
       Mini       0.22      0.26      0.85      0.24      0.47      0.20       477
     Teacup       0.68      0.37      0.85      0.47      0.56      0.30      1492
        Toy       0.14      0.26      0.83      0.18      0.47      0.20       316
         XL       0.52      0.70      0.93      0.60      0.81      0.64       315

avg / total       0.47      0.38      0.87      0.40      0.57      0.32      3285



-------------------------

In [65]:
# Remove loan status target from features data
target=df.fire_size_bin
y=target
# X = df.loc[:, df.columns !=target]
X=df.drop(columns=['fire_size_bin']).values

# Split training/test datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [66]:
# Train the EasyEnsembleClassifier
from imblearn.ensemble import EasyEnsembleClassifier
model = EasyEnsembleClassifier(n_estimators=100, random_state=1) 

#Fitting the model
model = model.fit(X_train, y_train)

# Making predictions using the testing data.
y_pred = model.predict(X_test)

In [67]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
acc_score = balanced_accuracy_score(y_test, y_pred)

In [69]:
from sklearn.metrics import confusion_matrix
# Calculating the confusion matrix
cm = confusion_matrix(y_test, y_pred)
cm_df = pd.DataFrame(
cm, index=["Actual Teacup", "Actual Toy", "Actual Mini", "Actual Medium", "Actual Large", "Actual XL"], columns=["Predicted Teacup", "Predicted Toy", "Predicted Mini", "Predicted Medium", "Predicted Large", "Predicted XL"]
)
cm_df

,Predicted Teacup,Predicted Toy,Predicted Mini,Predicted Medium,Predicted Large,Predicted XL
Actual Teacup,140,52,3,14,6,187
Actual Toy,46,72,55,57,35,18
Actual Mini,37,77,122,146,81,14
Actual Medium,49,185,298,740,195,25
Actual Large,12,38,75,131,54,6
Actual XL,95,12,4,3,3,198


In [70]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

      Large       0.37      0.35      0.92      0.36      0.57      0.30       402
     Medium       0.17      0.25      0.88      0.20      0.47      0.21       283
       Mini       0.22      0.26      0.85      0.24      0.46      0.20       477
     Teacup       0.68      0.50      0.80      0.57      0.63      0.39      1492
        Toy       0.14      0.17      0.89      0.16      0.39      0.14       316
         XL       0.44      0.63      0.92      0.52      0.76      0.56       315

avg / total       0.46      0.40      0.85      0.42      0.57      0.33      3285



In [71]:
# Displaying Easy Ensemble Classifier
print("Balanced Random Forest Clasifier")
print("Confusion Matrix")
display(cm_df)
print(f"Accuracy Score : {acc_score}")
print("Classification Report")
print(classification_report_imbalanced(y_test, y_pred))

Balanced Random Forest Clasifier
Confusion Matrix


,Predicted Teacup,Predicted Toy,Predicted Mini,Predicted Medium,Predicted Large,Predicted XL
Actual Teacup,140,52,3,14,6,187
Actual Toy,46,72,55,57,35,18
Actual Mini,37,77,122,146,81,14
Actual Medium,49,185,298,740,195,25
Actual Large,12,38,75,131,54,6
Actual XL,95,12,4,3,3,198


Accuracy Score : 0.3589794872599075
Classification Report
                   pre       rec       spe        f1       geo       iba       sup

      Large       0.37      0.35      0.92      0.36      0.57      0.30       402
     Medium       0.17      0.25      0.88      0.20      0.47      0.21       283
       Mini       0.22      0.26      0.85      0.24      0.46      0.20       477
     Teacup       0.68      0.50      0.80      0.57      0.63      0.39      1492
        Toy       0.14      0.17      0.89      0.16      0.39      0.14       316
         XL       0.44      0.63      0.92      0.52      0.76      0.56       315

avg / total       0.46      0.40      0.85      0.42      0.57      0.33      3285

